## Import libraries

In [1]:
import psycopg2
import pandas as pd

## Create functions 

In [2]:
def create_database(dbn_new):
        #connect to default database
        conn=psycopg2.connect(dbname="postgres",host="localhost",user="postgres",password="harsh")
        conn.set_session(autocommit=True)
        cur=conn.cursor()
        
        #create new database
        cur.execute(f"DROP DATABASE IF EXISTS {dbn_new};")
        cur.execute(f"CREATE DATABASE {dbn_new};")
        
        #close connection
        conn.close()
        
        # reconnect to the newly created database
        conn=psycopg2.connect(dbname=f"{dbn_new}",host="localhost",user="postgres",password="harsh")
        cur=conn.cursor()
        
        return cur,conn

In [3]:
def drop_table(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_table(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
AccountsCountry=pd.read_csv('Wealth-AccountsCountry.csv')
AccountsCountry.head(2)

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina


In [6]:
AccountsCountry_clean=AccountsCountry[['Code','Short Name','Table Name','Long Name','Currency Unit']]
AccountsCountry_clean.head(2)

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso


In [7]:
AccountsData=pd.read_csv('Wealth-AccountData.csv')
AccountsData.head(2)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68


In [8]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

##### Renaming Column

In [9]:
AccountsData.columns=['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995', '1996', '1997', '1998','1999', '2000', '2001', '2002','2003', '2004', '2005', '2006','2007', '2008', '2009', '2010','2011', '2012', '2013', '2014','2015', '2016', '2017', '2018']

##### New cleaned data to be inserted

In [10]:
AccountsData_clean=AccountsData[['Country Name', 'Country Code', 'Series Name', 'Series Code','1995','2000','2005','2010','2014']]

In [11]:
AccountsSeries=pd.read_csv('Wealth-AccountSeries.csv')
AccountsSeries.head(2)

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [12]:
AccountsSeries.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [13]:
AccountsSeries_clean=AccountsSeries[['Code','Topic','Previous Indicator Code','Previous Indicator Name']]

AccountsSeries_clean.head()

,Code,Topic,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital,NaN,NaN
1,NW.HCA.PC,Human capital,NaN,NaN
2,NW.HCA.FEMP.PC,Human capital,NaN,NaN
3,NW.HCA.MEMP.PC,Human capital,NaN,NaN
4,NW.HCA.FEMA.PC,Human capital,NaN,NaN


In [14]:
AccountsSeries_clean.rename(columns={'Code':'Series Code'})

,Series Code,Topic,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital,NaN,NaN
1,NW.HCA.PC,Human capital,NaN,NaN
2,NW.HCA.FEMP.PC,Human capital,NaN,NaN
3,NW.HCA.MEMP.PC,Human capital,NaN,NaN
4,NW.HCA.FEMA.PC,Human capital,NaN,NaN
5,NW.HCA.MALE.PC,Human capital,NaN,NaN
6,NW.HCA.FSEM.PC,Human capital,NaN,NaN
7,NW.HCA.MSEM.PC,Human capital,NaN,NaN
8,NW.HCA.FEMP.TO,Human capital,NaN,NaN
9,NW.HCA.MEMP.TO,Human capital,NaN,NaN


## CREATE DATABASE

In [15]:
cur,conn=create_database('accounts')

## CREATE TABLES

In [16]:
AccountsCountry_clean.columns

Index(['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit'], dtype='object')

In [17]:
accountscountry_table_create=("""CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR ,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
);""")
cur.execute(accountscountry_table_create)
conn.commit()

In [18]:
accountsdata_table_create=("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 NUMERIC,
year_2000 NUMERIC,
year_2005 NUMERIC,
year_2010 NUMERIC,
year_2014 NUMERIC
);""")
cur.execute(accountsdata_table_create)
conn.commit()

In [19]:
accountseries_table_create=("""CREATE TABLE IF NOT EXISTS accountseries(
series_code VARCHAR,
topic VARCHAR,
previous_indicator_code VARCHAR,
previous_indicator_name VARCHAR
);""")
cur.execute(accountseries_table_create)
conn.commit()

## INSERT DATA

In [20]:
accountscountry_data_insert=("""INSERT INTO accountscountry(
country_code,
short_name ,
table_name ,
long_name ,
currency_unit )
VALUES (%s,%s,%s,%s,%s)
""")

In [21]:
for i,row in AccountsCountry_clean.iterrows():
    cur.execute(accountscountry_data_insert,list(row))

In [22]:
accounts_data_table_insert = ("""
INSERT INTO accountsdata (
    country_name,
    country_code,
    indicator_name,
    indicator_code,
    year_1995,
    year_2000,
    year_2005,
    year_2010,
    year_2014
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")


In [23]:

import numpy as np

for _, row in AccountsData_clean.iterrows():
    # Replace non-numeric values with None
    row = row.apply(lambda x: x if str(x).replace('.', '').isdigit() else np.nan)
    
    # Convert the row to a list
    row_values = row.values.tolist()

    # Execute the SQL query
    cur.execute(accounts_data_table_insert, row_values)


In [24]:
accountseries_table_insert=("""INSERT INTO accountseries(
series_code,
topic,
previous_indicator_code,
previous_indicator_name
)
VALUES(%s,%s,%s,%s)
""")

In [25]:
for i,row in AccountsSeries_clean.iterrows():
    cur.execute(accountseries_table_insert,list(row))

In [26]:
conn.commit()

## VERIFY TABLE CREATION

In [27]:
cur.execute("SELECT * FROM accountscountry")

In [28]:
cur.fetchone()


('ALB', 'Albania', 'Albania', 'Republic of Albania', 'Albanian lek')